In [1]:
import os
import numpy as np
import scipy

from setup_paths import *
import visu_common
import measurement

mems = visu_common.get_mems(MEASUREMENTS_PATH)
directions = ['s', 'r']
meas_type = 'latency'

identified_params = {}
for mem in mems:
    for direction in directions:
        # here mem is the folder
        clocks = visu_common.get_clocks_in_folder(
            os.path.join(MEASUREMENTS_PATH, mem),
            prefix=f'meas_{direction}_', clock_lambda=lambda m7, m4: m4 >= 60)

        K_list = [] # matrix for the linear regression, 
        y_list = []
        for i, (m7, m4) in enumerate(clocks):
            dir_prefix = os.path.join(MEASUREMENTS_PATH, mem,
                                      f'meas_{direction}_{m7}_{m4}')
            sizes = sorted(visu_common.get_sizes(dir_prefix, size_lambda=lambda s: True))
            # row vector of measurements for sizes
            y = measurement.get_and_calc_meas(m4, dir_prefix, sizes, meas_type)
            y_list.append(y[0, :]) # mean and column
            K = np.hstack((1/m7*np.ones((len(sizes), 1)), # const part for sending
                        1/m7*np.array(sizes).reshape(-1, 1), # size dependent part for sending
                        1/m4*np.array(sizes).reshape(-1, 1),
                        1/m4*np.ones((len(sizes), 1))))
            K_list.append(K)
            
        K_matrix = np.vstack(K_list)
        y_vector = np.hstack(y_list) # to column vector

        res = scipy.optimize.lsq_linear(K_matrix, y_vector, bounds=(0, np.inf))
        print(res.message)
        # direction under mem in the dict
        if mem in identified_params:
            identified_params[mem][direction] = list(res.x)
        else:
            identified_params[mem] = {direction: list(res.x)}

The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.
The unconstrained solution is optimal.


In [6]:
import json

# Writing the result to file
model_path = os.path.join(MEASUREMENTS_PATH, 'models_long.json')
with open(model_path, 'w') as file:
    json.dump(identified_params, file, indent=4)

In [2]:
print(identified_params)

{'D1': {'s': [1486.570274249926, 1.7548721648276562, 17.24149474496164, 4065.7778256745382], 'r': [2070.272755333509, 0.9122070304884353, 21.936315727596977, 3925.3498818438748]}, 'D1_idcache_mpu_ncacheable': {'s': [1182.4215698185028, 0.020687474910685566, 17.173270539610463, 3208.3130785318494], 'r': [2013.273084485079, 4.371506849919694, 19.933482545901697, 2156.8871523949615]}, 'D2': {'s': [1896.7327360833033, 2.2233230800377886, 12.223636355735017, 3597.260692039512], 'r': [1903.3245551420462, 4.905945466821722, 12.783284527198948, 3780.2402180705635]}, 'D2_icache': {'s': [1275.6711992294672, 3.4740439767743663, 11.444140137232534, 3778.6951597414127], 'r': [1829.644713063877, 4.8811903335682985, 12.766132031581837, 3335.0579092957882]}, 'D2_idcache_mpu_ncacheable': {'s': [1286.9674962558627, 2.856506043874644, 8.183183544622075, 2996.621416844765], 'r': [1982.523732708172, 4.74438593533417, 13.5807362528117, 1931.0064218103284]}, 'D2_idcache_mpu_wrth_nalloc': {'s': [1424.93502409